In [ ]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2, remove_semantics, get_merge_pcds, merge_unite_gt
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil
from tqdm import tqdm 

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 6
old = False 

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [ ]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [ ]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )
        #o3d.visualization.draw_geometries([pcd_nonground_minor])


In [ ]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

In [ ]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

Now we subsample the poses based on a voxel_size

In [ ]:
if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz') == False : 
	print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz')
	poses, positions, \
	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder)



In [ ]:
with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_0.npz') as data:
	poses=data['poses']
	positions=data['positions']
	sampled_indices_local = data['sampled_indices_local']
	sampled_indices_global=data['sampled_indices_global']

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
overlap = 3
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
    if semantics : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors)
    else : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/255)
    return pcd_colored

In [18]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0
tarl_norm = False
ncuts_threshold = 0.03
        
out_name = 'sam_only' + str(SEQUENCE_NUM) 
# Generate 30 different colors
 

COLORS = generate_random_colors(400)
COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]


out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'

if os.path.exists(out_kitti_instance) == True : 
        shutil.rmtree(out_kitti_instance)
os.makedirs(out_kitti_instance)


out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'
'''
if os.path.exists(out_kitti_semantic) == True : 
        shutil.rmtree(out_kitti_semantic)
os.makedirs(out_kitti_semantic)
'''

limit = -1 ##use this for experiments to run limit chunks numberss


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))

instances = np.hstack((kitti_labels_orig['instance_nonground'].reshape(-1,),kitti_labels_orig['instance_ground'].reshape(-1,)))
                
merge_pcd = o3d.geometry.PointCloud()    

print("total sequence number",len(center_ids))
for sequence in tqdm(range(0,len(center_ids))):
        if NCUT_ground == False : 
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,list(sampled_indices_global),
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold,obb=obbs[sequence])
                

                
                
                
                seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                
                file_name = str(center_ids[sequence]).zfill(6) + '.pcd'

                kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True
                                        )
                
                kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                        colors=COLORS,gt_labels=semantics,semantics=True)
                
                #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                #semantic_labels = np.hstack(( kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                #np.savez(out_kitti_semantic + file_name.split('.')[0] + '.npz',labels=semantic_labels)
                
                o3d.io.write_point_cloud(out_folder_ncuts + file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                print('write gt',out_kitti_instance + file_name)
                o3d.io.write_point_cloud(out_kitti_instance + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                #o3d.io.write_point_cloud(out_kitti_semantic + file_name, kitti_chunk + kitti_chunk_semantic_ground, write_ascii=False, compressed=False, print_progress=False)
                
                


        
        
        

In [ ]:
out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

point_clouds = get_merge_pcds(out_folder_ncuts)[:-1]
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
#point_clouds_kitti_semantic = get_merge_pcds(out_kitti_semantic)[:-1]
merge = merge_chunks_unite_instances2(point_clouds)


#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)

#merge_kitti = merge_unite_gt(point_clouds_kitti)
merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
#merge_kitti_semantic = merge_unite_gt(point_clouds_kitti_semantic)

#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_semantic7.pcd", merge_kitti_semantic, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + out_name + str(SEQUENCE_NUM) + ".pcd", merge, write_ascii=False, compressed=False, print_progress=False)
print(merge_kitti_instance)
print(merge)
o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)


In [ ]:

merge = o3d.io.read_point_cloud(out_folder + out_name + str(SEQUENCE_NUM) +  ".pcd")
unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd")
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
print(merge_kitti_instance)



print('remove semantics')
new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)


metrics_ncuts = Metrics(name='ncuts')

metrics_ncuts.update_stats(labels_ncuts,new_ncuts_labels,labels_kitti)

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])


In [ ]:
merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
o3d.visualization.draw_geometries([merge_vis])